# SpinSystem Examples

Documentation: http://pycav.readthedocs.io/en/latest/api/quantum/spinsystem.html

The `SpinSystem` class creates the Hamiltonian of a group of interacting spins. To initialise the class, a list of spins (e.g. `[0.5, 1.5, 1.0]`) and an array of couplings (e.g. `[[0,1,1],[1,0,1],[0,1,1]]` for all-to-all coupling) must be passed as arguments to `SpinSystem`.

The spin-spin coupling in the Hamiltonian is of the form

$$ H = - J_{ij} \vec{S_i} \otimes \vec{S_j}, $$

with summation over $i$ and $j$. The coupling array used to initialise `SpinSystem` has elements $J_{ij}$.

In [1]:
from pycav.quantum import SpinSystem
import numpy as np
import matplotlib.pyplot as plt

# create a triangle of spins (1/2, 1/2, 1)
spins = [0.5, 0.5, 1.0]
couplings = [[0,1,1],[1,0,1],[1,1,0]]
spin_triangle = SpinSystem(spins, couplings)

# calculate the energy levels
spin_triangle.get_energies()
print("Energy levels of the spin triangle")
print(spin_triangle.energies)

Energy levels of the spin triangle
[-1.25 -1.25 -1.25 -1.25 -1.25  0.75  0.75  0.75  0.75  0.75  0.75  1.75]


In [2]:
# alternatively, find the multiplicities
# and non-repeated energies
spin_triangle.count_multiplicities()
print(spin_triangle.multiplicities)
print(spin_triangle.reduced_energies)

[5, 6, 1]
[-1.25, 0.75000000000000022, 1.7500000000000009]


## Magnetic Field

A magnetic field can also be passed to `SpinSystem` on initialisation, giving terms in the Hamiltonian of the form

$$ H = - \vec{B} . \vec{S} .$$

By default, $ \vec{B} = 0 $. The B-field is passed as a list of three-dimensional vectors, each corresponding to a different spin. The magnetic moments are set to unity, but their effects can be replicated by passing different magnetic fields to each spin.

In the example below, applying a magnetic field splits the energies of the triplet states of a pair of spin-1/2 particles.

In [3]:
# create the familiar spin-1/2 pair and find the energy levels
spins = [0.5, 0.5]
couplings = [[0,1],[1,0]]

spin_pair1 = SpinSystem(spins, couplings)
spin_pair1.get_energies()
print("Energy levels with no magnetic field")
print(spin_pair1.energies)

# then apply a magnetic field in the x-direction to both spins
B= [[0.1,0.0,0.0], [0.1,0.0,0.0]]
spin_pair2 = SpinSystem(spins, couplings, B_field = B)
spin_pair2.get_energies()
print("Energy level in a homogeneous magnetic field")
print(spin_pair2.energies)

Energy levels with no magnetic field
[-0.25 -0.25 -0.25  0.75]
Energy level in a homogeneous magnetic field
[-0.35 -0.25 -0.15  0.75]


## Anisotropy

In a ring of spins, it's likely that the couplings between the components of spin which are parallel and perpendicular to the ring will be different. In this example, we model a ring of 6 spins in which there is only $S_x-S_x$ coupling. To do this, it's necessary to use the `scaling` parameter when initialising our `SpinSystem`.

In [25]:
N = 6
spins = [0.5 for i in range(N)]

#coupling between adjacent spins only
couplings = np.zeros((N,N))
for i in range(N):
    for j in range(N):
        if abs(i-j) == 1 or abs(i-j) == N-1:
            couplings[i,j] = 1.0

#coupling between x components only            
scaling = [1.0, 0.0, 0.0]

spin_ring = SpinSystem(spins, couplings, scaling = scaling)
spin_ring.count_multiplicities()
print("Multiplicity, Energy")
for i in range(len(spin_ring.multiplicities)):
    print(spin_ring.multiplicities[i], ", %.2f" % spin_ring.reduced_energies[i])



Multiplicity, Energy
2 , -1.50
30 , -0.50
30 , 0.50
2 , 1.50


## Splitting of Energy Levels in a B-field
Consider the splitting of the hydrogen $2p$ level. The spin of the electron is $S = 1/2$ so we can treat this as a system of two coupled angular momenta, $1/2$ and $1$, each independently coupled to the magnetic field.
However, the isolated electron g-factor is approximately twice the orbital g-factor. In order to capture this, we must apply a stronger $B$-field to the $S=1/2$.

In [44]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

angular_momenta = [0.5, 1.0]
couplings = [[0,-1],[-1,0]]

B_increment = 0.04
fig = figure(title = "Hydrogen 2p Splitting", x_axis_label = "Magnetic Field", y_axis_label = "Energy")
for i in range(100):
    B = np.array([0,0,0.05*i])
    pair = SpinSystem(angular_momenta, couplings, B_field = [2*B,B])
    pair.count_multiplicities()
    fig.scatter([i*B_increment for energy in pair.reduced_energies], pair.reduced_energies, color = 'indigo')

show(fig)

Loading BokehJS ...